In [1]:
%pip install arxiv

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6105 sha256=4d09e4f251ffe0ba86cd8bd18d7eba8e1d65845df4e978f9d941a903bcfb9b24
  Stored in directory: c:\users\amolj\appdata\local\pip\cache\wheels\03\f5\1a\23761066dac1d0e8e683e5fdb27e12de53209d05a4a37e6246
Successfully built sgmllib3k


In [4]:
%pip install wikipedia

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11787 sha256=adab7241e77cbc41b16feebf73755d972b1ce21441224e38eda5e65323ba9826
  Stored in directory: c:\users\amolj\appdata\local\pip\cache\wheels\63\47\7c\a9688349aa74d228ce0a9023229c6c0ac52ca2a40fe87679b8
Successfully built wikipedia
Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [21]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200)
wiki_tool = WikipediaQueryRun(api_wrapper=api_wrapper)

In [9]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_weaviate.vectorstores import WeaviateVectorStore
from weaviate.auth import Auth
import os
import weaviate

loader = WebBaseLoader('https://docs.smith.langchain.com/')

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [10]:
docs = loader.load_and_split(text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200))

In [11]:
client = weaviate.connect_to_weaviate_cloud(
            cluster_url=f'https://{os.getenv("WEAVIATE_URL")}',
            auth_credentials=Auth.api_key(os.getenv('WEAVIATE_API_KEY')),
            headers={
                    'X-OpenAI-Api-Key': os.getenv('OPENAI_API_KEY') # openai api key for vectorizer & generative
                }
        )

open_ai_embeddings = OpenAIEmbeddings()

In [13]:
db_conection = WeaviateVectorStore(client=client, index_name="Langsmith", text_key="content", embedding=open_ai_embeddings)

In [14]:
db_conection.add_documents(docs)

['249f4417-e7e2-48e2-984a-b1d5c9a18d73',
 'e6d3355d-fd1d-43fd-a1a5-a6e27275344d',
 '426f9df7-56ae-4763-94b4-dc1baf64ec01',
 '3089a3fd-c8ba-4bf1-a617-42b4101157bf',
 'fdedaa08-5d97-4219-aa47-3a08b477a43c']

In [16]:
retriver = db_conection.as_retriever(
            search_type="mmr",  # or "similarity"
            search_kwargs={"k": 5, "lambda_mult": 0.7}
            )

In [17]:
from langchain.tools.retriever import create_retriever_tool
retriver_tool = create_retriever_tool(retriver, 'langsmith_search', 'Search for information about the langsmith. for any questions about Langsmith you must use this tool.')

In [20]:
## Arxiv Tool
from langchain_community.utilities.arxiv import ArxivAPIWrapper
from langchain_community.tools.arxiv import ArxivQueryRun

arxiv_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv_tool = ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv_tool.name

'arxiv'

In [22]:
tools = [wiki_tool, retriver_tool,arxiv_tool]

In [23]:
from dotenv import load_dotenv

load_dotenv()

from langchain_openai.chat_models import ChatOpenAI

openai_llm = ChatOpenAI(temperature=0)

In [35]:
# fetch prompt from langsmith Hub
from langsmith import Client
client = Client(api_key=os.getenv('LANGSMITH_API_KEY'))
prompt = client.pull_prompt("hwchase17/openai-tools-agent")
prompt.messages


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [32]:
## Agents
from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(openai_llm, tools, prompt=prompt)

In [33]:
## Agent Executor

from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [34]:
agent_executor.invoke({'input': 'Tell me about Langsmith'})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'Langsmith'}`


Get started with LangSmith | 🦜️🛠️ LangSmith

LangSmith + LangChain OSSLangSmith is framework-agnostic — it can be used with or without LangChain's open source frameworks langchain and langgraph.If you are using either of these, you can enable LangSmith tracing with a single environment variable.
For more see the how-to guide for setting up LangSmith with LangChain or setting up LangSmith with LangGraph.
Observability​
Observability is important for any software application, but especially so for LLM applications. LLMs are non-deterministic by nature, meaning they can produce unexpected results. This makes them trickier than normal to debug.
This is where LangSmith can help! LangSmith has LLM-native observability, allowing you to get meaningful insights from your application. LangSmith’s observability features have you covered throughout all stages of application development - from prot

{'input': 'Tell me about Langsmith',
 'output': 'LangSmith is a platform for building production-grade Large Language Models (LLM) applications. It allows you to closely monitor and evaluate your application, enabling you to ship quickly and with confidence. LangSmith provides observability features, evaluation capabilities, and tools for prompt engineering to help you optimize and improve your LLM applications.\n\nObservability:\n- Analyze traces in LangSmith and configure metrics, dashboards, and alerts based on these traces.\n- Evaluate your application over production traffic to score application performance and receive human feedback on your data.\n\nEvals:\n- Use the LangSmith SDK and UI to build and run high-quality evaluations for your applications.\n- Quickly assess the performance of your application using off-the-shelf evaluators.\n- Analyze evaluation results in the LangSmith UI and compare results over time.\n- Collect human feedback on your data to enhance your applicatio

In [36]:
agent_executor.invoke({'input': 'what the new paper on attention models'})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': 'attention models'}`




d:\personal\llm_pocs\venv\Lib\site-packages\langchain_community\utilities\arxiv.py:105: DeprecationWarning: The 'Search.results' method is deprecated, use 'Client.results' instead
  ).results()


Published: 2017-09-19
Title: Exploring Human-like Attention Supervision in Visual Question Answering
Authors: Tingting Qiao, Jianfeng Dong, Duanqing Xu
Summary: Attention mechanisms have been widely aThe new paper on attention models is titled "Exploring Human-like Attention Supervision in Visual Question Answering" by Tingting Qiao, Jianfeng Dong, and Duanqing Xu. The paper explores human-like attention supervision in visual question answering.

> Finished chain.


{'input': 'what the new paper on attention models',
 'output': 'The new paper on attention models is titled "Exploring Human-like Attention Supervision in Visual Question Answering" by Tingting Qiao, Jianfeng Dong, and Duanqing Xu. The paper explores human-like attention supervision in visual question answering.'}

In [38]:
agent_executor.invoke({'input': 'who is current chamipons of football'})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'current champions of football'}`


Page: List of English football champions
Summary: The English football champions are the winners of the top-level league in English men's football, which since the 1992–93 season has been called the PThe current champions of English football are Manchester City.

> Finished chain.


{'input': 'who is current chamipons of football',
 'output': 'The current champions of English football are Manchester City.'}